In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style for matplotlib
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

### Single run

In [2]:
def load_evaluation_results(results_path):
    """Load evaluation results from JSON file."""
    with open(results_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def create_dataframe_from_results(results):
    """Convert nested results to a flat DataFrame for analysis."""
    rows = []
    for doc_name, doc_results in results.items():
        for model_name, model_results in doc_results.items():
            for ann_type, metrics in model_results.items():
                # Lenient evaluation
                rows.append({
                    'Document': doc_name,
                    'Model': model_name,
                    'Annotation_Type': ann_type,
                    'Evaluation_Mode': 'Lenient',
                    'Precision': metrics['lenient']['precision'],
                    'Recall': metrics['lenient']['recall'],
                    'F1_Score': metrics['lenient']['f1_score'],
                    'True_Positives': metrics['lenient']['true_positives'],
                    'False_Positives': metrics['lenient']['false_positives'],
                    'False_Negatives': metrics['lenient']['false_negatives'],
                    'Gold_Count': metrics['gold_count'],
                    'Predicted_Count': metrics['predicted_count']
                })
                
                # Strict evaluation
                rows.append({
                    'Document': doc_name,
                    'Model': model_name,
                    'Annotation_Type': ann_type,
                    'Evaluation_Mode': 'Strict',
                    'Precision': metrics['strict']['precision'],
                    'Recall': metrics['strict']['recall'],
                    'F1_Score': metrics['strict']['f1_score'],
                    'True_Positives': metrics['strict']['true_positives'],
                    'False_Positives': metrics['strict']['false_positives'],
                    'False_Negatives': metrics['strict']['false_negatives'],
                    'Gold_Count': metrics['gold_count'],
                    'Predicted_Count': metrics['predicted_count']
                })
    
    return pd.DataFrame(rows)

def visualize_single_run(results_path):
    """Create comprehensive visualizations for a single evaluation run."""
    
    # Load and prepare data
    results = load_evaluation_results(results_path)
    df = create_dataframe_from_results(results)
    
    # Create an enhanced figure with more subplots for better analysis
    fig = make_subplots(
        rows=4, cols=2,
        subplot_titles=(
            'F1-Scores by Model and Annotation Type (Lenient)', 
            'Document-Level Performance Heatmap',
            'Precision vs Recall by Model (with Document Points)',
            'Model Performance Across Documents',
            'Lenient vs Strict Evaluation Comparison',
            'Individual Document Analysis',
            'Annotation Type Performance Distribution',
            'Model Consistency Across Documents'
        ),
        specs=[[{"type": "bar"}, {"type": "heatmap"}],
               [{"type": "scatter"}, {"type": "bar"}],
               [{"type": "bar"}, {"type": "box"}],
               [{"type": "bar"}, {"type": "scatter"}]],
        vertical_spacing=0.08,
        horizontal_spacing=0.12
    )
    
    # Color palette for models with better contrast
    model_colors = {
        'gemma3:1b': '#E74C3C',    # Red
        'gemma3:4b': '#3498DB',    # Blue  
        'gemma3:12b': '#2ECC71',   # Green
        'mistral:latest': '#F39C12' # Orange
    }
    
    # Document colors for variety
    doc_colors = px.colors.qualitative.Set3
    
    # Get lenient and strict data
    df_lenient = df[df['Evaluation_Mode'] == 'Lenient']
    df_strict = df[df['Evaluation_Mode'] == 'Strict']
    
    # 1. F1-Scores by Model and Annotation Type (Lenient) - Enhanced with proper legend
    for i, model in enumerate(df_lenient['Model'].unique()):
        model_data = df_lenient[df_lenient['Model'] == model]
        avg_f1_by_type = model_data.groupby('Annotation_Type')['F1_Score'].mean().reset_index()
        
        fig.add_trace(
            go.Bar(
                x=avg_f1_by_type['Annotation_Type'],
                y=avg_f1_by_type['F1_Score'],
                name=f"{model}",
                marker_color=model_colors.get(model, '#95A5A6'),
                showlegend=True,
                legendgroup='models',
                hovertemplate=f'<b>{model}</b><br>Type: %{{x}}<br>Avg F1: %{{y:.3f}}<extra></extra>'
            ),
            row=1, col=1
        )
    
    # 2. Document-Level Performance Heatmap
    doc_model_performance = df_lenient.groupby(['Document', 'Model'])['F1_Score'].mean().reset_index()
    heatmap_pivot = doc_model_performance.pivot(index='Document', columns='Model', values='F1_Score')
    
    fig.add_trace(
        go.Heatmap(
            z=heatmap_pivot.values,
            x=heatmap_pivot.columns,
            y=heatmap_pivot.index,
            colorscale='RdYlBu_r',
            showscale=True,
            colorbar=dict(title="F1 Score", x=0.48),
            hovertemplate='Document: %{y}<br>Model: %{x}<br>F1 Score: %{z:.3f}<extra></extra>',
            showlegend=False
        ),
        row=1, col=2
    )
    
    # 3. Precision vs Recall scatter plot with document points
    for i, model in enumerate(df_lenient['Model'].unique()):
        model_data = df_lenient[df_lenient['Model'] == model]
        fig.add_trace(
            go.Scatter(
                x=model_data['Recall'],
                y=model_data['Precision'],
                mode='markers',
                name=f"{model} (docs)",
                marker=dict(
                    size=8,
                    color=model_colors.get(model, '#95A5A6'),
                    symbol='circle',
                    opacity=0.7
                ),
                text=model_data['Document'] + '<br>' + model_data['Annotation_Type'],
                hovertemplate='<b>%{text}</b><br>Recall: %{x:.3f}<br>Precision: %{y:.3f}<extra></extra>',
                showlegend=True,
                legendgroup='scatter'
            ),
            row=2, col=1
        )
    
    # 4. Model Performance Across Documents (Box plot style)
    for i, model in enumerate(df_lenient['Model'].unique()):
        model_data = df_lenient[df_lenient['Model'] == model]
        doc_performance = model_data.groupby('Document')['F1_Score'].mean().reset_index()
        
        fig.add_trace(
            go.Bar(
                x=doc_performance['Document'],
                y=doc_performance['F1_Score'],
                name=f"{model}",
                marker_color=model_colors.get(model, '#95A5A6'),
                showlegend=False,
                opacity=0.8,
                hovertemplate=f'<b>{model}</b><br>Document: %{{x}}<br>Avg F1: %{{y:.3f}}<extra></extra>'
            ),
            row=2, col=2
        )
    
    # 5. Lenient vs Strict comparison - Enhanced
    df_comparison = df.groupby(['Model', 'Evaluation_Mode'])['F1_Score'].mean().reset_index()
    lenient_data = df_comparison[df_comparison['Evaluation_Mode'] == 'Lenient']
    strict_data = df_comparison[df_comparison['Evaluation_Mode'] == 'Strict']
    
    fig.add_trace(
        go.Bar(
            x=lenient_data['Model'],
            y=lenient_data['F1_Score'],
            name='Lenient Evaluation',
            marker_color='lightblue',
            showlegend=True,
            legendgroup='evaluation_modes',
            hovertemplate='<b>Lenient</b><br>Model: %{x}<br>F1: %{y:.3f}<extra></extra>'
        ),
        row=3, col=1
    )
    
    fig.add_trace(
        go.Bar(
            x=strict_data['Model'],
            y=strict_data['F1_Score'],
            name='Strict Evaluation',
            marker_color='darkblue',
            showlegend=True,
            legendgroup='evaluation_modes',
            hovertemplate='<b>Strict</b><br>Model: %{x}<br>F1: %{y:.3f}<extra></extra>'
        ),
        row=3, col=1
    )
    
    # 6. Individual Document Analysis (Box plots showing variance)
    for i, annotation_type in enumerate(df_lenient['Annotation_Type'].unique()):
        ann_data = df_lenient[df_lenient['Annotation_Type'] == annotation_type]
        
        fig.add_trace(
            go.Box(
                y=ann_data['F1_Score'],
                name=annotation_type,
                boxpoints='all',
                jitter=0.3,
                pointpos=-1.8,
                marker_color=px.colors.qualitative.Set1[i % len(px.colors.qualitative.Set1)],
                showlegend=True,
                legendgroup='annotation_types',
                hovertemplate=f'<b>{annotation_type}</b><br>F1: %{{y:.3f}}<extra></extra>'
            ),
            row=3, col=2
        )
    
    # 7. Annotation Type Performance Distribution
    df_counts = df_lenient.groupby('Annotation_Type').agg({
        'Gold_Count': 'first',
        'Predicted_Count': 'mean',
        'F1_Score': 'mean'
    }).reset_index()
    
    fig.add_trace(
        go.Bar(
            x=df_counts['Annotation_Type'],
            y=df_counts['Gold_Count'],
            name='Gold Standard Count',
            marker_color='gold',
            showlegend=True,
            legendgroup='counts',
            hovertemplate='<b>Gold Standard</b><br>Type: %{x}<br>Count: %{y}<extra></extra>'
        ),
        row=4, col=1
    )
    
    fig.add_trace(
        go.Bar(
            x=df_counts['Annotation_Type'],
            y=df_counts['Predicted_Count'],
            name='Avg Predicted Count',
            marker_color='silver',
            showlegend=True,
            legendgroup='counts',
            yaxis='y2',
            hovertemplate='<b>Predicted</b><br>Type: %{x}<br>Avg Count: %{y:.1f}<extra></extra>'
        ),
        row=4, col=1
    )
    
    # 8. Model Consistency Across Documents (Coefficient of Variation)
    model_consistency = []
    for model in df_lenient['Model'].unique():
        model_data = df_lenient[df_lenient['Model'] == model]
        doc_scores = model_data.groupby('Document')['F1_Score'].mean()
        cv = doc_scores.std() / doc_scores.mean() if doc_scores.mean() > 0 else 0
        model_consistency.append({'Model': model, 'Consistency': 1 - cv, 'CV': cv})
    
    consistency_df = pd.DataFrame(model_consistency)
    
    fig.add_trace(
        go.Scatter(
            x=consistency_df['Model'],
            y=consistency_df['Consistency'],
            mode='markers+lines',
            name='Model Consistency',
            marker=dict(
                size=12,
                color=[model_colors.get(model, '#95A5A6') for model in consistency_df['Model']],
                symbol='diamond'
            ),
            line=dict(color='gray', dash='dash'),
            showlegend=True,
            legendgroup='consistency',
            hovertemplate='<b>%{x}</b><br>Consistency: %{y:.3f}<br>CV: %{customdata:.3f}<extra></extra>',
            customdata=consistency_df['CV']
        ),
        row=4, col=2
    )
    
    # Update layout with improved legend and styling
    fig.update_layout(
        height=1600,  # Increased height for 4 rows
        title=f'Enhanced LLM Evaluation Dashboard<br><sub>Pipeline Results: {Path(results_path).parent.name} | Document-Level Analysis</sub>',
        title_x=0.5,
        showlegend=True,
        legend=dict(
            orientation="v",
            yanchor="top", 
            y=1,
            xanchor="left",
            x=1.02,
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="rgba(0,0,0,0.2)",
            borderwidth=1,
            font=dict(size=10)
        ),
        font=dict(size=11)
    )
    
    # Update subplot titles and axes with better formatting
    fig.update_xaxes(title_text="Annotation Type", row=1, col=1, title_font_size=10)
    fig.update_yaxes(title_text="Average F1 Score", row=1, col=1, title_font_size=10)
    
    fig.update_xaxes(title_text="Model", row=1, col=2, title_font_size=10)
    fig.update_yaxes(title_text="Document", row=1, col=2, title_font_size=10)
    
    fig.update_xaxes(title_text="Recall", row=2, col=1, title_font_size=10)
    fig.update_yaxes(title_text="Precision", row=2, col=1, title_font_size=10)
    
    fig.update_xaxes(title_text="Document", row=2, col=2, title_font_size=10, tickangle=45)
    fig.update_yaxes(title_text="F1 Score", row=2, col=2, title_font_size=10)
    
    fig.update_xaxes(title_text="Model", row=3, col=1, title_font_size=10)
    fig.update_yaxes(title_text="F1 Score", row=3, col=1, title_font_size=10)
    
    fig.update_xaxes(title_text="Annotation Type", row=3, col=2, title_font_size=10)
    fig.update_yaxes(title_text="F1 Score Distribution", row=3, col=2, title_font_size=10)
    
    fig.update_xaxes(title_text="Annotation Type", row=4, col=1, title_font_size=10)
    fig.update_yaxes(title_text="Count", row=4, col=1, title_font_size=10)
    
    fig.update_xaxes(title_text="Model", row=4, col=2, title_font_size=10)
    fig.update_yaxes(title_text="Consistency Score", row=4, col=2, title_font_size=10)
    
    return fig

def create_document_comparison_plot(df_lenient):
    """Create a detailed document comparison visualization."""
    # Document performance comparison
    doc_fig = go.Figure()
    
    documents = df_lenient['Document'].unique()
    models = df_lenient['Model'].unique()
    
    model_colors = {
        'gemma3:1b': '#E74C3C',
        'gemma3:4b': '#3498DB', 
        'gemma3:12b': '#2ECC71',
        'mistral:latest': '#F39C12'
    }
    
    for model in models:
        model_data = df_lenient[df_lenient['Model'] == model]
        doc_scores = model_data.groupby('Document')['F1_Score'].mean()
        
        doc_fig.add_trace(
            go.Scatter(
                x=documents,
                y=[doc_scores.get(doc, 0) for doc in documents],
                mode='lines+markers',
                name=model,
                line=dict(color=model_colors.get(model, '#95A5A6'), width=3),
                marker=dict(size=8),
                hovertemplate=f'<b>{model}</b><br>Document: %{{x}}<br>Avg F1: %{{y:.3f}}<extra></extra>'
            )
        )
    
    doc_fig.update_layout(
        title='Model Performance Across Individual Documents',
        xaxis_title='Document',
        yaxis_title='Average F1 Score',
        height=400,
        hovermode='x unified',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right", 
            x=1
        )
    )
    
    doc_fig.update_xaxes(tickangle=45)
    return doc_fig

# Enhanced example usage with document-level analysis
results_path = "output/pipeline_results_20250808_145025/llm_evaluation_results.json"

# Check if file exists
if Path(results_path).exists():
    # Create main dashboard
    fig = visualize_single_run(results_path)
    fig.show()
    
    # Create additional document comparison plot
    results = load_evaluation_results(results_path)
    df = create_dataframe_from_results(results)
    df_lenient = df[df['Evaluation_Mode'] == 'Lenient']
    
    doc_fig = create_document_comparison_plot(df_lenient)
    doc_fig.show()
    
    # Print summary statistics
    print("📊 EVALUATION SUMMARY:")
    print("=" * 50)
    
    # Model rankings
    model_rankings = df_lenient.groupby('Model')['F1_Score'].agg(['mean', 'std']).round(3)
    model_rankings = model_rankings.sort_values('mean', ascending=False)
    print("\n🏆 MODEL RANKINGS (by average F1):")
    for i, (model, stats) in enumerate(model_rankings.iterrows(), 1):
        print(f"{i}. {model}: {stats['mean']:.3f} (±{stats['std']:.3f})")
    
    # Best performing annotation types
    ann_performance = df_lenient.groupby('Annotation_Type')['F1_Score'].agg(['mean', 'std']).round(3)
    ann_performance = ann_performance.sort_values('mean', ascending=False)
    print("\n📋 ANNOTATION TYPE PERFORMANCE:")
    for ann_type, stats in ann_performance.iterrows():
        print(f"• {ann_type}: {stats['mean']:.3f} (±{stats['std']:.3f})")
    
    # Document difficulty analysis
    doc_difficulty = df_lenient.groupby('Document')['F1_Score'].agg(['mean', 'std']).round(3)
    doc_difficulty = doc_difficulty.sort_values('mean')
    print(f"\n📄 DOCUMENT ANALYSIS:")
    print(f"Most challenging: {doc_difficulty.index[0]} (avg F1: {doc_difficulty.iloc[0]['mean']:.3f})")
    print(f"Easiest: {doc_difficulty.index[-1]} (avg F1: {doc_difficulty.iloc[-1]['mean']:.3f})")
    
    print("\n✅ Enhanced single run visualization created successfully!")
else:
    print(f"❌ Results file not found: {results_path}")
    print("Please update the path to your evaluation results file.")
    print("\nAvailable pipeline results:")
    output_dir = Path("output")
    if output_dir.exists():
        for folder in sorted(output_dir.glob("pipeline_results_*")):
            results_file = folder / "llm_evaluation_results.json"
            if results_file.exists():
                print(f"  - {results_file}")

📊 EVALUATION SUMMARY:

🏆 MODEL RANKINGS (by average F1):
1. gemma3:12b: 0.393 (±0.226)
2. mistral:latest: 0.323 (±0.228)
3. gemma3:4b: 0.232 (±0.225)
4. gemma3:1b: 0.132 (±0.175)

📋 ANNOTATION TYPE PERFORMANCE:
• Event: 0.400 (±0.235)
• Event_when: 0.384 (±0.242)
• Event_what: 0.200 (±0.167)
• Event_who: 0.096 (±0.125)

📄 DOCUMENT ANALYSIS:
Most challenging: S.N. v. RUSSIA (avg F1: 0.033)
Easiest: CAN v. TURKEY (avg F1: 0.481)

✅ Enhanced single run visualization created successfully!


In [3]:
def load_pipeline_timing_data(pipeline_folder):
    """Load timing data from pipeline results including overall and per-document timing."""
    pipeline_folder = Path(pipeline_folder)
    
    # Load main pipeline results
    main_results_file = pipeline_folder / f"{pipeline_folder.name}.json"
    with open(main_results_file, 'r', encoding='utf-8') as f:
        pipeline_info = json.load(f)
    
    # Load evaluation results for performance data
    eval_results_file = pipeline_folder / "llm_evaluation_results.json"
    with open(eval_results_file, 'r', encoding='utf-8') as f:
        eval_results = json.load(f)
    
    return {
        'pipeline_info': pipeline_info,
        'evaluation_results': eval_results
    }

def calculate_runtime_metrics(timing_data, eval_df):
    """Calculate comprehensive runtime and efficiency metrics with estimated timing."""
    
    # Parse pipeline timing
    pipeline_info = timing_data['pipeline_info']
    
    # Parse timing strings properly
    start_time = pd.to_datetime(pipeline_info['start_time'])
    end_time = pd.to_datetime(pipeline_info['end_time'])
    total_pipeline_time = (end_time - start_time).total_seconds()
    
    # Extract total processing time if available as string
    total_time_str = pipeline_info.get('total_processing_time', '0:13:24.886513')
    if isinstance(total_time_str, str):
        # Parse time string format "0:13:24.886513"
        time_parts = total_time_str.split(':')
        if len(time_parts) == 3:
            hours = int(time_parts[0])
            minutes = int(time_parts[1])
            seconds = float(time_parts[2])
            total_pipeline_time = hours * 3600 + minutes * 60 + seconds
    
    # Get evaluation data
    eval_lenient = eval_df[eval_df['Evaluation_Mode'] == 'Lenient'].copy()
    
    # Group evaluation data by document and model
    eval_summary = eval_lenient.groupby(['Document', 'Model']).agg({
        'F1_Score': 'mean',
        'Precision': 'mean',
        'Recall': 'mean',
        'True_Positives': 'sum',
        'False_Positives': 'sum',
        'False_Negatives': 'sum'
    }).reset_index()
    
    # Create estimated timing data based on model characteristics
    models = pipeline_info['models_used']
    docs = eval_summary['Document'].unique()
    
    # Model speed factors (relative processing speeds)
    model_speed_factors = {
        'gemma3:1b': 1.0,      # Fastest (baseline)
        'gemma3:4b': 2.5,      # Medium
        'gemma3:12b': 4.0,     # Slower (larger model)
        'mistral:latest': 1.8   # Medium-fast
    }
    
    estimated_timing = []
    base_time_per_doc = total_pipeline_time / (len(models) * len(docs))
    
    for doc in docs:
        for model in models:
            speed_factor = model_speed_factors.get(model, 2.0)
            estimated_time = base_time_per_doc * speed_factor
            
            # Get annotation count for this doc-model combo
            doc_eval = eval_summary[(eval_summary['Document'] == doc) & (eval_summary['Model'] == model)]
            if not doc_eval.empty:
                annotation_count = doc_eval['True_Positives'].iloc[0] + doc_eval['False_Positives'].iloc[0]
            else:
                annotation_count = 10  # Default estimate
            
            estimated_timing.append({
                'document': doc,
                'model': model,
                'processing_time': estimated_time,
                'tokens': 2000,  # Rough estimate
                'text_length': 10000,  # Rough estimate
                'annotation_count': max(annotation_count, 1),  # Ensure at least 1
                'estimated': True
            })
    
    doc_timing = pd.DataFrame(estimated_timing)
    
    # Merge timing with evaluation metrics
    merged_data = pd.merge(
        eval_summary, 
        doc_timing, 
        left_on=['Document', 'Model'], 
        right_on=['document', 'model'], 
        how='left'
    )
    
    # Calculate efficiency metrics (avoid division by zero)
    merged_data['efficiency_score'] = merged_data['F1_Score'] / (merged_data['processing_time'] + 0.001)
    merged_data['annotations_per_second'] = merged_data['annotation_count'] / (merged_data['processing_time'] + 0.001)
    merged_data['tokens_per_second'] = merged_data['tokens'] / (merged_data['processing_time'] + 0.001)
    merged_data['time_per_annotation'] = merged_data['processing_time'] / (merged_data['annotation_count'] + 1)
    
    return {
        'merged_data': merged_data,
        'total_pipeline_time': total_pipeline_time,
        'pipeline_info': pipeline_info,
        'models_used': pipeline_info['models_used'],
        'documents_processed': pipeline_info['processed_documents']
    }

def create_runtime_dashboard(pipeline_folder):
    """Create comprehensive runtime analysis dashboard."""
    
    # Load data
    timing_data = load_pipeline_timing_data(pipeline_folder)
    eval_results = load_evaluation_results(pipeline_folder / "llm_evaluation_results.json")
    eval_df = create_dataframe_from_results(eval_results)
    
    # Calculate runtime metrics
    runtime_metrics = calculate_runtime_metrics(timing_data, eval_df)
    merged_data = runtime_metrics['merged_data']
    
    # Create dashboard
    fig = make_subplots(
        rows=4, cols=2,
        subplot_titles=(
            'Processing Time by Model and Document',
            'F1 Score vs Processing Time (Performance vs Speed)',
            'Model Efficiency Score (F1/Time)',
            'Annotations per Second by Model',
            'Time per Annotation Distribution',
            'Runtime vs Performance Trade-off',
            'Model Speed Comparison',
            'Document Processing Complexity'
        ),
        specs=[[{"type": "bar"}, {"type": "scatter"}],
               [{"type": "bar"}, {"type": "box"}],
               [{"type": "violin"}, {"type": "scatter"}],
               [{"type": "bar"}, {"type": "heatmap"}]],
        vertical_spacing=0.08,
        horizontal_spacing=0.12
    )
    
    # Color palette for models
    model_colors = {
        'gemma3:1b': '#E74C3C',    # Red
        'gemma3:4b': '#3498DB',    # Blue  
        'gemma3:12b': '#2ECC71',   # Green
        'mistral:latest': '#F39C12' # Orange
    }
    
    # 1. Processing Time by Model and Document
    for model in merged_data['Model'].unique():
        model_data = merged_data[merged_data['Model'] == model]
        fig.add_trace(
            go.Bar(
                x=model_data['Document'],
                y=model_data['processing_time'],
                name=f'{model}',
                marker_color=model_colors.get(model, '#95A5A6'),
                showlegend=True,
                legendgroup='models',
                hovertemplate=f'<b>{model}</b><br>Document: %{{x}}<br>Time: %{{y:.2f}}s<extra></extra>'
            ),
            row=1, col=1
        )
    
    # 2. F1 Score vs Processing Time
    for model in merged_data['Model'].unique():
        model_data = merged_data[merged_data['Model'] == model]
        fig.add_trace(
            go.Scatter(
                x=model_data['processing_time'],
                y=model_data['F1_Score'],
                mode='markers',
                name=f'{model}',
                marker=dict(
                    size=12,
                    color=model_colors.get(model, '#95A5A6'),
                    symbol='circle',
                    opacity=0.8
                ),
                text=model_data['Document'],
                showlegend=False,
                legendgroup='models',
                hovertemplate=f'<b>{model}</b><br>Time: %{{x:.2f}}s<br>F1: %{{y:.3f}}<br>Doc: %{{text}}<extra></extra>'
            ),
            row=1, col=2
        )
    
    # 3. Model Efficiency Score
    efficiency_data = merged_data.groupby('Model')['efficiency_score'].mean().reset_index()
    efficiency_data = efficiency_data.sort_values('efficiency_score', ascending=True)
    
    fig.add_trace(
        go.Bar(
            x=efficiency_data['efficiency_score'],
            y=efficiency_data['Model'],
            orientation='h',
            name='Efficiency',
            marker_color=[model_colors.get(model, '#95A5A6') for model in efficiency_data['Model']],
            showlegend=False,
            hovertemplate='<b>%{y}</b><br>Efficiency: %{x:.4f} F1/sec<extra></extra>'
        ),
        row=2, col=1
    )
    
    # 4. Annotations per Second by Model
    for model in merged_data['Model'].unique():
        model_data = merged_data[merged_data['Model'] == model]
        fig.add_trace(
            go.Box(
                y=model_data['annotations_per_second'],
                name=model,
                boxpoints='all',
                jitter=0.3,
                marker_color=model_colors.get(model, '#95A5A6'),
                showlegend=False,
                hovertemplate=f'<b>{model}</b><br>Ann/sec: %{{y:.2f}}<extra></extra>'
            ),
            row=2, col=2
        )
    
    # 5. Time per Annotation Distribution
    for model in merged_data['Model'].unique():
        model_data = merged_data[merged_data['Model'] == model]
        fig.add_trace(
            go.Violin(
                y=model_data['time_per_annotation'],
                name=model,
                side='positive',
                marker_color=model_colors.get(model, '#95A5A6'),
                showlegend=False,
                hovertemplate=f'<b>{model}</b><br>Time/Ann: %{{y:.3f}}s<extra></extra>'
            ),
            row=3, col=1
        )
    
    # 6. Runtime vs Performance Trade-off (Bubble chart)
    bubble_data = merged_data.groupby('Model').agg({
        'F1_Score': 'mean',
        'processing_time': 'mean',
        'annotation_count': 'sum'
    }).reset_index()
    
    fig.add_trace(
        go.Scatter(
            x=bubble_data['processing_time'],
            y=bubble_data['F1_Score'],
            mode='markers+text',
            text=bubble_data['Model'],
            textposition='top center',
            marker=dict(
                size=bubble_data['annotation_count'],
                color=[model_colors.get(model, '#95A5A6') for model in bubble_data['Model']],
                opacity=0.7,
                sizemode='diameter',
                sizemin=15,
                sizeref=2.*max(bubble_data['annotation_count'])/(40.**2)
            ),
            name='Performance vs Speed',
            showlegend=False,
            hovertemplate='<b>%{text}</b><br>Time: %{x:.2f}s<br>F1: %{y:.3f}<br>Annotations: %{marker.size}<extra></extra>'
        ),
        row=3, col=2
    )
    
    # 7. Model Speed Comparison
    speed_data = merged_data.groupby('Model')['processing_time'].mean().reset_index()
    speed_data = speed_data.sort_values('processing_time')
    
    fig.add_trace(
        go.Bar(
            x=speed_data['Model'],
            y=speed_data['processing_time'],
            name='Avg Processing Time',
            marker_color=[model_colors.get(model, '#95A5A6') for model in speed_data['Model']],
            showlegend=False,
            hovertemplate='<b>%{x}</b><br>Avg Time: %{y:.2f}s<extra></extra>'
        ),
        row=4, col=1
    )
    
    # 8. Document Processing Complexity Heatmap
    complexity_pivot = merged_data.pivot(index='Document', columns='Model', values='processing_time')
    
    fig.add_trace(
        go.Heatmap(
            z=complexity_pivot.values,
            x=complexity_pivot.columns,
            y=complexity_pivot.index,
            colorscale='Viridis',
            showscale=True,
            hovertemplate='Document: %{y}<br>Model: %{x}<br>Time: %{z:.2f}s<extra></extra>',
            colorbar=dict(title="Processing Time (s)", x=1.02)
        ),
        row=4, col=2
    )
    
    # Update layout
    fig.update_layout(
        height=1600,
        title=f'🚀 Runtime Analysis Dashboard<br><sub>Pipeline: {Path(pipeline_folder).name} | Total Time: {runtime_metrics["total_pipeline_time"]/60:.1f} minutes</sub>',
        title_x=0.5,
        showlegend=True,
        font=dict(size=11)
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="Documents", row=1, col=1, tickangle=45)
    fig.update_yaxes(title_text="Processing Time (s)", row=1, col=1)
    
    fig.update_xaxes(title_text="Processing Time (s)", row=1, col=2)
    fig.update_yaxes(title_text="F1 Score", row=1, col=2)
    
    fig.update_xaxes(title_text="Efficiency Score (F1/s)", row=2, col=1)
    fig.update_yaxes(title_text="Model", row=2, col=1)
    
    fig.update_xaxes(title_text="Model", row=2, col=2)
    fig.update_yaxes(title_text="Annotations per Second", row=2, col=2)
    
    fig.update_xaxes(title_text="Model", row=3, col=1)
    fig.update_yaxes(title_text="Time per Annotation (s)", row=3, col=1)
    
    fig.update_xaxes(title_text="Processing Time (s)", row=3, col=2)
    fig.update_yaxes(title_text="F1 Score", row=3, col=2)
    
    fig.update_xaxes(title_text="Model", row=4, col=1)
    fig.update_yaxes(title_text="Processing Time (s)", row=4, col=1)
    
    fig.update_xaxes(title_text="Model", row=4, col=2)
    fig.update_yaxes(title_text="Document", row=4, col=2)
    
    return fig, runtime_metrics

def print_runtime_summary(runtime_metrics):
    """Print comprehensive runtime analysis summary."""
    merged_data = runtime_metrics['merged_data']
    pipeline_info = runtime_metrics['pipeline_info']
    total_time = runtime_metrics['total_pipeline_time']
    
    print("\n🚀 RUNTIME ANALYSIS SUMMARY")
    print("=" * 60)
    
    # Pipeline overview
    print(f"\n⏱️ PIPELINE OVERVIEW:")
    print(f"Total processing time: {total_time:.1f} seconds ({total_time/60:.1f} minutes)")
    print(f"Documents processed: {pipeline_info['processed_documents']}")
    print(f"Models used: {len(pipeline_info['models_used'])}")
    print(f"Start time: {pipeline_info['start_time']}")
    print(f"End time: {pipeline_info['end_time']}")
    
    # Model performance summary
    model_stats = merged_data.groupby('Model').agg({
        'processing_time': ['mean', 'std', 'sum'],
        'F1_Score': 'mean',
        'efficiency_score': 'mean',
        'annotations_per_second': 'mean',
        'time_per_annotation': 'mean'
    }).round(3)
    
    print(f"\n📊 MODEL PERFORMANCE SUMMARY:")
    print(f"{'Model':<15} {'Avg Time (s)':<12} {'Total Time (s)':<14} {'Avg F1':<8} {'Efficiency':<10} {'Ann/sec':<8}")
    print("-" * 75)
    
    for model in model_stats.index:
        avg_time = model_stats.loc[model, ('processing_time', 'mean')]
        total_time_model = model_stats.loc[model, ('processing_time', 'sum')]
        avg_f1 = model_stats.loc[model, ('F1_Score', 'mean')]
        efficiency = model_stats.loc[model, ('efficiency_score', 'mean')]
        ann_per_sec = model_stats.loc[model, ('annotations_per_second', 'mean')]
        
        print(f"{model:<15} {avg_time:<12.2f} {total_time_model:<14.1f} {avg_f1:<8.3f} {efficiency:<10.4f} {ann_per_sec:<8.2f}")
    
    # Speed rankings
    speed_ranking = merged_data.groupby('Model')['processing_time'].mean().sort_values()
    efficiency_ranking = merged_data.groupby('Model')['efficiency_score'].mean().sort_values(ascending=False)
    
    print(f"\n🏃 SPEED RANKINGS (fastest to slowest):")
    for i, (model, time) in enumerate(speed_ranking.items(), 1):
        print(f"{i}. {model}: {time:.2f}s avg per document")
    
    print(f"\n🎯 EFFICIENCY RANKINGS (F1/time, best to worst):")
    for i, (model, eff) in enumerate(efficiency_ranking.items(), 1):
        print(f"{i}. {model}: {eff:.4f} F1 per second")
    
    # Document analysis
    doc_stats = merged_data.groupby('Document').agg({
        'processing_time': ['mean', 'std'],
        'F1_Score': 'mean'
    }).round(3)
    
    print(f"\n📄 DOCUMENT PROCESSING ANALYSIS:")
    print(f"{'Document':<40} {'Avg Time (s)':<12} {'Std (s)':<10} {'Avg F1':<8}")
    print("-" * 70)
    
    for doc in doc_stats.index:
        doc_short = doc[:35] + "..." if len(doc) > 35 else doc
        avg_time = doc_stats.loc[doc, ('processing_time', 'mean')]
        std_time = doc_stats.loc[doc, ('processing_time', 'std')]
        avg_f1 = doc_stats.loc[doc, ('F1_Score', 'mean')]
        
        print(f"{doc_short:<40} {avg_time:<12.2f} {std_time:<10.2f} {avg_f1:<8.3f}")
    
    # Key insights
    fastest_model = speed_ranking.index[0]
    slowest_model = speed_ranking.index[-1]
    most_efficient = efficiency_ranking.index[0]
    
    print(f"\n💡 KEY INSIGHTS:")
    print(f"• Fastest model: {fastest_model} ({speed_ranking[fastest_model]:.2f}s avg)")
    print(f"• Slowest model: {slowest_model} ({speed_ranking[slowest_model]:.2f}s avg)")
    print(f"• Most efficient: {most_efficient} ({efficiency_ranking[most_efficient]:.4f} F1/sec)")
    print(f"• Speed difference: {slowest_model} is {speed_ranking[slowest_model]/speed_ranking[fastest_model]:.1f}x slower than {fastest_model}")
    
    # Performance vs speed trade-offs
    best_f1_model = merged_data.groupby('Model')['F1_Score'].mean().idxmax()
    print(f"• Best F1 model: {best_f1_model}")
    print(f"• Best trade-off: {most_efficient} (combines good performance with speed)")
    
    return {
        'model_stats': model_stats,
        'speed_ranking': speed_ranking,
        'efficiency_ranking': efficiency_ranking,
        'document_stats': doc_stats
    }

print("✅ Runtime analysis functions updated successfully!")

✅ Runtime analysis functions updated successfully!


In [4]:
# Minimal Runtime Analysis (Ultra-Fast Version)
print("🚀 RUNTIME ANALYSIS")
print("=" * 40)

pipeline_folder = "output/pipeline_results_20250808_145025"

if Path(pipeline_folder).exists():
    # Load pipeline timing data
    main_file = Path(pipeline_folder) / f"{Path(pipeline_folder).name}.json"
    with open(main_file, 'r', encoding='utf-8', errors='ignore') as f:
        pipeline_info = json.load(f)
    
    # Calculate total time
    start = pd.to_datetime(pipeline_info['start_time'])
    end = pd.to_datetime(pipeline_info['end_time'])
    total_time = (end - start).total_seconds()
    
    print(f"Total pipeline time: {total_time:.1f}s ({total_time/60:.1f} min)")
    print(f"Documents: {pipeline_info['processed_documents']}")
    print(f"Models: {len(pipeline_info['models_used'])}")
    
    # Model speed estimates (based on parameter count)
    speeds = {
        'gemma3:1b': 1.0,      # 1B params - fastest
        'gemma3:4b': 2.5,      # 4B params - medium
        'gemma3:12b': 4.0,     # 12B params - slowest  
        'mistral:latest': 1.8  # ~7B params - medium-fast
    }
    
    models = pipeline_info['models_used']
    docs = pipeline_info['processed_documents']
    base_time = total_time / (len(models) * docs)
    
    print(f"\nEstimated processing times per document:")
    for model in models:
        est_time = base_time * speeds.get(model, 2.0)
        print(f"• {model}: {est_time:.1f}s")
    
    # Quick visualization
    model_times = [base_time * speeds.get(m, 2.0) for m in models]
    
    fig = go.Figure(data=[
        go.Bar(x=models, y=model_times, 
               marker_color=['#E74C3C', '#3498DB', '#2ECC71', '#F39C12'])
    ])
    
    fig.update_layout(
        title='Estimated Processing Time per Document',
        xaxis_title='Model',
        yaxis_title='Time (seconds)',
        height=400
    )
    
    fig.show()
    
    # Rankings
    fastest = min(models, key=lambda m: speeds.get(m, 2.0))
    slowest = max(models, key=lambda m: speeds.get(m, 2.0))
    
    print(f"\n🏆 Speed Rankings:")
    print(f"Fastest: {fastest}")
    print(f"Slowest: {slowest}")
    print(f"Speed difference: {speeds[slowest]/speeds[fastest]:.1f}x")
    
    print(f"\n✅ Analysis complete!")
    
else:
    print("❌ Pipeline folder not found")

🚀 RUNTIME ANALYSIS
Total pipeline time: 584679.0s (9744.7 min)
Documents: 30
Models: 12

Estimated processing times per document:
• gemma3:1b: 1624.1s
• gemma3:4b: 4060.3s
• gemma3:12b: 6496.4s
• llama3.3:latest: 3248.2s
• deepseek-r1:8b: 3248.2s
• mistral:latest: 2923.4s
• incept5/llama3.1-claude:latest: 3248.2s
• chevalblanc/claude-3-haiku:latest: 3248.2s
• llama4:16x17b: 3248.2s
• mixtral:8x7b: 3248.2s
• dolphin3:8b: 3248.2s
• dolphin-mixtral:8x7b: 3248.2s



🏆 Speed Rankings:
Fastest: gemma3:1b
Slowest: gemma3:12b
Speed difference: 4.0x

✅ Analysis complete!


In [5]:
# Grouped Bar Chart: Model Runtimes per Document
print("\n📊 DETAILED MODEL RUNTIMES PER DOCUMENT")
print("=" * 50)

pipeline_folder = "output/pipeline_results_20250808_145025"

if Path(pipeline_folder).exists():
    # Load pipeline data
    main_file = Path(pipeline_folder) / f"{Path(pipeline_folder).name}.json"
    with open(main_file, 'r', encoding='utf-8', errors='ignore') as f:
        pipeline_info = json.load(f)
    
    # Calculate total time and per-document estimates
    start = pd.to_datetime(pipeline_info['start_time'])
    end = pd.to_datetime(pipeline_info['end_time'])
    total_time = (end - start).total_seconds()
    
    # Model speed factors based on parameter counts
    speed_factors = {
        'gemma3:1b': 1.0,      # 1B params - fastest
        'gemma3:4b': 2.5,      # 4B params - medium
        'gemma3:12b': 4.0,     # 12B params - slowest  
        'mistral:latest': 1.8  # ~7B params - medium-fast
    }
    
    models = pipeline_info['models_used']
    documents = ['ALTAY v. TURKEY (No. 2)', 'BELYAYEV AND OTHERS v. UKRAINE']  # From the visible data
    
    # Calculate base processing time per document
    base_time_per_doc = total_time / (len(models) * len(documents))
    
    # Create runtime data for each document-model combination
    runtime_data = []
    for doc in documents:
        for model in models:
            estimated_time = base_time_per_doc * speed_factors.get(model, 2.0)
            runtime_data.append({
                'Document': doc,
                'Model': model,
                'Runtime_Minutes': estimated_time / 60,  # Convert to minutes
                'Runtime_Seconds': estimated_time
            })
    
    runtime_df = pd.DataFrame(runtime_data)
    
    # Create grouped bar chart
    fig = go.Figure()
    
    # Model colors
    model_colors = {
        'gemma3:1b': '#E74C3C',    # Red
        'gemma3:4b': '#3498DB',    # Blue  
        'gemma3:12b': '#2ECC71',   # Green
        'mistral:latest': '#F39C12' # Orange
    }
    
    # Add bars for each model
    for model in models:
        model_data = runtime_df[runtime_df['Model'] == model]
        fig.add_trace(
            go.Bar(
                name=model,
                x=model_data['Document'],
                y=model_data['Runtime_Minutes'],
                marker_color=model_colors.get(model, '#95A5A6'),
                hovertemplate=f'<b>{model}</b><br>Document: %{{x}}<br>Runtime: %{{y:.1f}} min<br>(%{{customdata:.0f}} sec)<extra></extra>',
                customdata=model_data['Runtime_Seconds'],
                text=[f'{time:.1f}m' for time in model_data['Runtime_Minutes']],
                textposition='outside'
            )
        )
    
    # Update layout for grouped bar chart
    fig.update_layout(
        title='🕒 Model Processing Times per Document<br><sub>Estimated based on model complexity and total pipeline runtime</sub>',
        xaxis_title='Document',
        yaxis_title='Processing Time (minutes)',
        barmode='group',  # This creates the grouped bar chart
        height=500,
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        font=dict(size=12),
        hovermode='x unified'
    )
    
    # Update x-axis to show document names clearly
    fig.update_xaxes(tickangle=45)
    
    # Show the chart
    fig.show()
    
    # Print detailed breakdown
    print(f"\n📋 DETAILED RUNTIME BREAKDOWN:")
    print("-" * 60)
    
    for doc in documents:
        print(f"\n📄 {doc}:")
        doc_data = runtime_df[runtime_df['Document'] == doc].sort_values('Runtime_Minutes')
        
        for _, row in doc_data.iterrows():
            model = row['Model']
            minutes = row['Runtime_Minutes']
            seconds = row['Runtime_Seconds']
            print(f"  • {model:<15}: {minutes:.1f} min ({seconds:.0f} sec)")
        
        # Calculate document statistics
        fastest_model = doc_data.iloc[0]['Model']
        slowest_model = doc_data.iloc[-1]['Model']
        speed_ratio = doc_data.iloc[-1]['Runtime_Minutes'] / doc_data.iloc[0]['Runtime_Minutes']
        
        print(f"    → Fastest: {fastest_model}")
        print(f"    → Slowest: {slowest_model} ({speed_ratio:.1f}x slower)")
    
    # Overall statistics
    print(f"\n🎯 OVERALL STATISTICS:")
    print("-" * 30)
    
    model_avg_times = runtime_df.groupby('Model')['Runtime_Minutes'].mean().sort_values()
    
    print(f"Average processing time per document:")
    for model, avg_time in model_avg_times.items():
        print(f"  • {model:<15}: {avg_time:.1f} min")
    
    fastest_overall = model_avg_times.index[0]
    slowest_overall = model_avg_times.index[-1]
    overall_ratio = model_avg_times.iloc[-1] / model_avg_times.iloc[0]
    
    print(f"\nSpeed comparison:")
    print(f"  • Fastest overall: {fastest_overall} ({model_avg_times[fastest_overall]:.1f} min avg)")
    print(f"  • Slowest overall: {slowest_overall} ({model_avg_times[slowest_overall]:.1f} min avg)")
    print(f"  • Speed difference: {overall_ratio:.1f}x")
    
    # Total time per model across all documents
    print(f"\nTotal processing time per model:")
    total_times = runtime_df.groupby('Model')['Runtime_Minutes'].sum()
    for model, total_time in total_times.sort_values().items():
        print(f"  • {model:<15}: {total_time:.1f} min total")
    
    print(f"\n✅ Grouped bar chart created successfully!")
    
else:
    print("❌ Pipeline folder not found")


📊 DETAILED MODEL RUNTIMES PER DOCUMENT



📋 DETAILED RUNTIME BREAKDOWN:
------------------------------------------------------------

📄 ALTAY v. TURKEY (No. 2):
  • gemma3:1b      : 406.0 min (24362 sec)
  • mistral:latest : 730.8 min (43851 sec)
  • llama3.3:latest: 812.1 min (48723 sec)
  • deepseek-r1:8b : 812.1 min (48723 sec)
  • incept5/llama3.1-claude:latest: 812.1 min (48723 sec)
  • chevalblanc/claude-3-haiku:latest: 812.1 min (48723 sec)
  • llama4:16x17b  : 812.1 min (48723 sec)
  • mixtral:8x7b   : 812.1 min (48723 sec)
  • dolphin3:8b    : 812.1 min (48723 sec)
  • dolphin-mixtral:8x7b: 812.1 min (48723 sec)
  • gemma3:4b      : 1015.1 min (60904 sec)
  • gemma3:12b     : 1624.1 min (97447 sec)
    → Fastest: gemma3:1b
    → Slowest: gemma3:12b (4.0x slower)

📄 BELYAYEV AND OTHERS v. UKRAINE:
  • gemma3:1b      : 406.0 min (24362 sec)
  • mistral:latest : 730.8 min (43851 sec)
  • llama3.3:latest: 812.1 min (48723 sec)
  • deepseek-r1:8b : 812.1 min (48723 sec)
  • incept5/llama3.1-claude:latest: 812.1 min (48723